In [1]:
import sklearn
import math
import re
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import collections
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error


In [12]:
   # io = Clustering.Clustring()
   # io.main()
io1 =  BOW()
# io1.getDatas()

#     p = Train_Test.TrainTest()
#     p.main()
# io1.loading_data()
evaluation = TrainTest()
evaluation.main()


Epoch 1/50
272/272 [==============================] - 1s 3ms/step - loss: 1.1204 - accuracy: 0.3967
Epoch 2/50
272/272 [==============================] - 1s 4ms/step - loss: 1.0557 - accuracy: 0.4400
Epoch 3/50
272/272 [==============================] - 1s 4ms/step - loss: 1.0253 - accuracy: 0.4733
Epoch 4/50
272/272 [==============================] - 1s 3ms/step - loss: 1.0046 - accuracy: 0.4888
Epoch 5/50
272/272 [==============================] - 1s 3ms/step - loss: 0.9913 - accuracy: 0.5162
Epoch 6/50
272/272 [==============================] - 1s 3ms/step - loss: 0.9814 - accuracy: 0.5257
Epoch 7/50
272/272 [==============================] - 1s 3ms/step - loss: 0.9761 - accuracy: 0.5279
Epoch 8/50
272/272 [==============================] - 1s 3ms/step - loss: 0.9714 - accuracy: 0.5377
Epoch 9/50
272/272 [==============================] - 1s 3ms/step - loss: 0.9640 - accuracy: 0.5445
Epoch 10/50
272/272 [==============================] - 1s 3ms/step - loss: 0.9602 - accuracy: 0.5487

ValueError: ignored

In [3]:

class BOW:
    numberOfTrain = np.empty((8696, 20))
    numberOfTest = np.empty((1587, 20))
    train_Cluster = []
    # we all of the comments of train and test files in these lists
    df1 = []
    df1_test = []
    # .........................we have al of te words in words list
    words = []
    numbers = []
    wordest = [[] for _ in range(0, 8709)]
    test_Wordest = [[] for _ in range(0, 1610)]
    vectors = []
    cluster_repitition = np.zeros((10))
    def getDatas(self):
        df = pd.read_csv("dataset/train.csv")
        self.comment_dataframe = df["Comment"]
        df2 = pd.read_csv("dataset/test.csv")
        self.comment_dataframe_test = df2["Comment"]

        self.make_Dic( self.comment_dataframe , self.comment_dataframe_test)



    def calculateBOW(self ,wordset,l_doc):
        tf_diz = dict.fromkeys(wordset,0)
        for word in l_doc:

            tf_diz[word]=l_doc.count(word)
        return tf_diz

#     if you run the function below you don not need to run it again
    def make_Dic(self , train_comments , test_comments):
        # in this function we extract the words which are exist in Word2Vec and the length of them is more than 3
        wv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


        wordest_train = []
        for i in range(0 ,len(train_comments) ):
            l_doc = re.sub(r"[^a-zA-Z]", " ", train_comments[i].lower()).split()
            wordest_train.append(l_doc)
        with open('wordest.txt', 'w') as f:
            for line in wordest_train:
                for word in line:
                    if(len(word) > 3 ) and wv.has_index_for(word) :
                        f.write(word)
                    f.write(" ")

                f.write('\n')

        wordest_test = []
        for i in range(0 ,len(test_comments) ):
            l_doc = re.sub(r"[^a-zA-Z]", " ", test_comments[i].lower()).split()
            wordest_test.append(l_doc)
        with open('wordest_test.txt', 'w') as f:
            for line in wordest_test:
                for word in line:
                    if(len(word) > 3 ) and wv.has_index_for(word):
                        f.write(word)
                    f.write(" ")

                f.write('\n')






#       in the loop below we delete those that are repeated more than once
        words = np.union1d(wordest_train[0] , wordest_train[1])
        for index in range(2 , len(train_comments)) :
            words = np.union1d(words , wordest_train[index])
        for index in range( 0 , len(test_comments)):
            words = np.union1d(words , wordest_test[index])

#          in the loop below, we eliminate the words which their lentgh is less than 4 due to the fact we guess these
#          words are prepositions
        lentgh = len(words)
        t = 0
        for index in range(0, lentgh):
            if len(words[index - t]) <= 3:
                words = np.delete(words, index - t)
                t += 1


#       in the loop below we delete the words which are not exist in Word2Vec list
        length= len(words)
        y = 0
        for io in range( 0 , length ):
            if not wv.has_index_for(words[io - y]):
                words = np.delete(words , io - y)
                y += 1
#       finally we write all of the words in a txt file
        with open('readme.txt', 'w') as f:
            for line in words:
                f.write(line)
                f.write('\n')


    def loading_data(self ) :
        df = pd.read_csv("dataset/train.csv")
        train_comments = df["Comment"]
        df2 = pd.read_csv("dataset/test.csv")
        test_comments = df2["Comment"]

        with open('readme.txt', 'r') as file:

            for line in file:
                for word in line.split() :
                    # displaying the words
                    self.words.append(word)

        with open('wordest.txt', 'r') as file:
            index = 0
            for line in file:
                for word in line.split():
                    # displaying the words
                    self.wordest[index].append(word)
                index +=1

        with open('wordest_test.txt', 'r') as file:
            index = 0
            for line in file:
                for word in line.split():
                    # displaying the words
                    self.test_Wordest[index].append(word)
                index += 1

        wv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


#     in this loop we vectroize the words with W2V
        for l in range(0 , len(self.words)) :
            vector = wv.get_vector(self.words[l])
            self.vectors.append(vector)

        kmeans = KMeans(n_clusters=20, random_state=0, n_init="auto").fit(self.vectors)


#     in this loop we count the number of each word in any cluster

        for index in range(0, len(train_comments)):
            for word in self.wordest[index] :
                self.numberOfTrain[index][kmeans.predict([wv.get_vector(word)])[0]] += 1
                print(index)

        for index in range(0 , len(test_comments)):
            for word in self.test_Wordest[index]:
                self.numberOfTrain[index][kmeans.predict([wv.get_vector(word)])[0]] += 1



        with open('numbersOfTrain.txt' , 'w' ) as f:
            for i in range(0 , len(self.numberOfTrain)):
                for j in range(0 , len(self.numberOfTrain[i])):
                    f.write(str(self.numberOfTrain[i][j]))
                    f.write(" ")
                f.write('\n')

        with open('numbersOfTest.txt', 'w') as f:
            for i in range(0, len(self.numberOfTest)):
                for j in range(0, len(self.numberOfTest[i])):
                    f.write(str(self.numberOfTest[i][j]))
                    f.write(" ")
                f.write('\n')



In [11]:
class TrainTest:

    train_Data = []
    train_Label = []

    test_Data=[]
    test_Label = []

    def main(self):

        with open('numbersOfTrain.txt', 'r') as file:
            for line in file:
                vec = []
                for word in line.split():
                    vec.append(float(word))
                self.train_Data.append(vec)
        self.train_Data.pop()
        with open('numbersOfTest.txt', 'r') as file:
            for line in file:
                vec = []
                for word in line.split():
                    vec.append(float(word))
                self.test_Data.append(vec)
        self.test_Data = self.test_Data.pop()
        df = pd.read_csv("dataset/train.csv")
        df = df["Topic"]
        for i in range(0 , len(df)):
            s = 0
            if df[i] == "Biology" :
                s = 0
            elif df[i] == ("Chemistry"):
                s = 1
            else:
                s = 2

            self.train_Label.append(s)

        df1 = pd.read_csv("dataset/test.csv")
        df1 = df1["Topic"]
        for i in range(0, len(df1)):
            s = 0
            if df[i]==("Biology"):
                s = 0
            elif df[i]==("Chemistry"):
                s = 1
            else:
                s = 2

            self.test_Label.append(s)



        model = Sequential()
        model.add(Dense(10, activation='relu', input_dim=20))
        model.add(Dense(5, activation='relu'))
        model.add(Dense(3, activation='sigmoid'))

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


        model.fit(self.train_Data, self.train_Label, epochs=50, batch_size=32)

        print(f' Final accuracy : {model.evaluate(self.test_Data , self.test_Label)}')

        # per = tree.DecisionTreeClassifier()
        # per = per.fit(self.train_Data, self.train_Label)

        # test = per.predict(self.test_Data)
        # result = accuracy_score(test, self.test_Label)
        # print(result)

#         clf = Perceptron(tol=1e-4, random_state=1)
#         clf.fit(self.train_Data, self.train_Label)
#         print(clf.score(self.train_Data, self.train_Label))
